In [1]:
from niffler_demo import Demo
from knowledgerepr import fieldnetwork
from modelstore.elasticstore import StoreHandler
from algebra import API
from ver_utils.column_selection import ColumnSelection

In [2]:
graph_path = '/home/cc/advw_graph/'
store_client = StoreHandler()
network = fieldnetwork.deserialize_network(graph_path)
aurum_api = API(network=network, store_client=store_client)

In [3]:
column_selection = ColumnSelection(aurum_api)
candidate_columns = column_selection.column_retreival(attr="", examples=['USA', 'China'])
print(candidate_columns)

{'2898653797': <ver_utils.column.Column object at 0x7f29fd601b00>, '4111388216': <ver_utils.column.Column object at 0x7f29fd56ad30>}
